In [310]:
import pandas as pd
import os
import datetime
import pymongo
import pprint
def extract_n_transform():
    """
    Part 1 will be to extract these data sources into one Pandas. Note to only extract the data that supports
    the types of questions you want. This is the Extract and Transform sections. For example, if you choose
    not to allow users to ask about the number of seasons, do not extract that into your new data set.
    """
    # Extract what I need
    #best_m_by_year = pd.read_csv(f"Best Movie by Year Netflix.csv").drop(columns=["index","MAIN_PRODUCTION"])
    best_m_overall =pd.read_csv(f"Best Movies Netflix.csv").drop(columns=["index","NUMBER_OF_VOTES","DURATION","MAIN_PRODUCTION"])
    #best_s_by_year = pd.read_csv(f"Best Show by Year Netflix.csv").drop(columns=["index","NUMBER_OF_SEASONS","MAIN_PRODUCTION"])
    best_s_overall = pd.read_csv(f"Best Shows Netflix.csv").drop(columns=["index","NUMBER_OF_VOTES","DURATION","NUMBER_OF_SEASONS","MAIN_PRODUCTION"])
    #raw_credits = pd.read_csv(f"raw_credits.csv").drop(columns=["index","person_id"])
    #raw_titles = pd.read_csv(f"raw_titles.csv").drop(columns=["index","type","release_year","age_certification","production_countries","seasons","imdb_id","imdb_score","imdb_votes","runtime", "genres"])

    # Transform the data
    MOVIES = best_m_overall
    SHOWS = best_s_overall
    #CREDITS = raw_credits.merge(raw_titles).rename(columns=str.lower)
    MOVIES["type"] = "movie"
    SHOWS["type"] = "show"

    M_n_S = pd.concat([MOVIES,SHOWS]).rename(columns=str.lower)
    #ALL_DATA=M_n_S.append(CREDITS)
    M_n_S = M_n_S.applymap(lambda s: s.lower() if type(s) == str else s)

    return M_n_S


In [311]:
# This is the extracted and formatted data
formated = extract_n_transform()
formated

,title,release_year,score,main_genre,type
0,david attenborough: a life on our planet,2020,9.0,documentary,movie
1,inception,2010,8.8,scifi,movie
2,forrest gump,1994,8.8,drama,movie
3,anbe sivam,2003,8.7,comedy,movie
4,bo burnham: inside,2021,8.7,comedy,movie
...,...,...,...,...,...
241,evil genius,2018,7.5,crime,show
242,13 reasons why,2017,7.5,drama,show
243,lupin,2021,7.5,crime,show
244,all of us are dead,2022,7.5,action,show


In [312]:
formated.main_genre.unique()

array(['documentary', 'scifi', 'drama', 'comedy', 'western', 'action',
       'crime', 'thriller', 'war', 'fantasy', 'romance', 'horror',
       'musical', 'animation', 'sports', 'reality'], dtype=object)

In [313]:
"""
Part 2 will be to Load that into a MongoDB. You code will create a new MongoDB and load the data into a Mongo Database
"""
def Load_data(df):
    data_dict = df.to_dict("records") # converts the data similar to JSON

    host_name = "localhost"
    port = "27017"
    conn_str = {
        "local" : f"mongodb://{host_name}:{port}/",
    }
    client = pymongo.MongoClient(conn_str["local"])
    print(f"Local Connection String: {conn_str['local']}")
    # Create the database
    db = client["Netflix"]
    user_info_table = db["Info"]
    user_info_table.insert_many(data_dict)
    return user_info_table

In [314]:
table = Load_data(formated)

Local Connection String: mongodb://localhost:27017/


## Here's what it looks like
![image](capture.png)

In [315]:
"""
Part 3 will be to run your bot to answer your questions. The bot will tell the user with that types of
questions they can ask when they enter “help”, and when the bot starts. If the user asks a question that
isn’t in the correct form, or isn’t in the pattern, it suggests the kinds of questions it can answer.

**You DO NOT need to make this bot run in Discord or Twitter, but rather at the local machine.**
I'm going to assume that means I can just use input()

I was going to use the basic bot code originally but I wasn't able to get good accuacy with my questions. 
"""
# Here's the list of items you can ask
Q_L = {"best movie": [ "best movie", "best movie?", "What movie is the best?", "Recommend me a movie"],
         "best show": ["best show", "best show?", "What movie is the show?", "Recommend me a show"],
         "worst movie": ["worst movie", "worst movie?", "What movie is the worst?", "bad movie", "bad movie"],
         "worst show": ["worst show", "worst show?", "What show is the worst", "bad show", "bad show?"],
         "best movie now": ["What is the best movie out now?", "best movie this year", "best movie this year?", "What movie is the best this year?", "Recommend me a movie from this year", "best movie now"],
         "best show now": ["What is the best show out now?", "best show this year", "best show this year?", "What show is best this year?", "Recommend me a show from this year", "best show now"],
         "worst movie now": ["What is the worst movie out now?", "worst movie this year", "worst movie now?", "What movie is the worst this year?", "bad movie this year", "bad movie this year", "worst movie now"],
         "worst show now": ["What is the worst show out now?", "worst show this year", "worst show this year?", "What show is the worst this year", "bad show now", "bad show now?", "worst show now"], 
         "best genre movie": ["What is the highest rated genre of movies on netflix", "What is the highest rated genre for movies?", "highest rated genre for movies?", "best genre movies","best genre movie"],
         "worst genre movie": ["What is the lowest rated genre of movies on netflix", "What is the lowest genre for movies?", "lowest rated genre for movies?", "worst genre movies", "worst genre movie"],
         "best genre show": ["What is the highest rated genre of shows on netflix", "What is the highest rated genre for shows?", "highest rated genre for shows?", "best genre shows"],
         "worst genre show": ["What is the lowest rated genre of shows on netflix", "What is the lowest genre for shows?", "lowest rated genre for shows?", "worst genre shows"],
         
       
       "best documentary movie": ["What is the highest rated documentary movie on netflix", "What is the highest rated documentary movie?", "highest rated documentary movie?", "best documentary movie", "what is the best documentary movie?", "what is the best documentary movie"],
         "best documentary show": ["What is the highest rated documentary show on netflix", "What is the highest rated documentary show?", "highest rated documentary show?", "best documentary show", "what is the best documentary show?", "what is the best documentary show"],
         
       "worst documentary movie": ["What is the lowest rated documentary movie on netflix", "What is the lowest rated documentary movie?", "lowest rated documentary movie?", "worst documentary movie", "what is the worst documentary movie?", "what is the worst documentary movie"],
         "worst documentary show": ["What is the lowest rated documentary show on netflix", "What is the lowest rated documentary show?", "lowest rated documentary show?", "worst documentary show", "what is the worst documentary show?", "what is the worst documentary show"],
         
       
       
        "best scifi movie": ["What is the highest rated scifi movie on netflix", "What is the highest rated scifi movie?", "highest rated scifi movie?", "best scifi movie", "What is the highest rated sci-fi movie", "What is the highest rated sci-fi movie?", "highest rated sci-fi movie?", "best sci-fi movie", "what is the best scifi movie?", "what is the best scifi movie"],
        "best scifi show": ["What is the highest rated scifi show on netflix", "What is the highest rated scifi show?", "highest rated scifi show?", "best scifi show", "What is the highest rated sci-fi show", "What is the highest rated sci-fi show?", "highest rated sci-fi? show", "best sci-fi show", "what is the best scifi show?", "what is the best scifi show"],
  
       "worst scifi movie": ["What is the lowest rated scifi movie on netflix", "What is the lowest rated scifi movie?", "lowest rated scifi movie?", "worst scifi movie", "What is the lowest rated sci-fi movie on netflix", "What is the lowest rated sci-fi movie?", "lowest rated sci-fi movie?", "worst sci-fi movie", "what is the worst scifi movie?", "what is the worst scifi movie"], 
        "worst scifi show": ["What is the lowest rated scifi show on netflix", "What is the lowest rated scifi show?", "lowest rated scifi show?", "worst scifi show", "What is the lowest rated sci-fi show on netflix", "What is the lowest rated sci-fi show?", "lowest rated sci-fi show?", "worst sci-fi show", "what is the worst scifi show?", "what is the worst scifi show"], 

       
       "best drama movie": ["What is the highest rated drama movie on netflix", "What is the highest rated drama movie?", "highest rated drama movie?", "best drama movie", "what is the best drama movie?", "what is the best drama movie"],
        "best drama show": ["What is the highest rated drama show on netflix", "What is the highest rated drama show?", "highest rated drama show?", "best drama show", "what is the best drama show?", "what is the best drama show"],
        
       "worst drama movie": ["What is the lowest rated drama movie on netflix", "What is the lowest rated drama movie?", "lowest rated drama movie?", "worst drama movie", "what is the worst drama movie?", "what is the worst drama movie"], 
       "worst drama show": ["What is the lowest rated drama show on netflix", "What is the lowest rated drama show?", "lowest rated drama?", "worst drama show", "what is the worst drama show?", "what is the worst drama show"], 
         
       
       "best comedy movie": ["What is the highest rated comedy movie on netflix", "What is the highest rated comedy movie?", "highest rated comedy movie?", "best comedy movie", "funniest comedy movie", "what is the best comedy movie?", "what is the best comedy movie"],
        "best comedy show": ["What is the highest rated comedy show on netflix", "What is the highest rated comedy show?", "highest rated comedy show?", "best comedy show", "funniest comedy show", "what is the best comedy show?", "what is the best comedy show"],
      
       "worst comedy movie": ["What is the lowest rated comedy movie on netflix", "What is the lowest rated comedy movie?", "lowest rated comedy movie?", "worst comedy movie", "what is the worst comedy movie?", "what is the worst comedy movie"],       
        "worst comedy show": ["What is the lowest rated comedy show on netflix", "What is the lowest rated comedy show?", "lowest rated comedy show?", "worst comedy show", "what is the worst comedy show?", "what is the worst comedy show"],       

       
       
       "best horror movie": ["What is the highest rated horror movie on netflix", "What is the highest rated horror movie?", "highest rated horror movie?", "best horror movie", "scariest horror movie", "what is the best horror movie?", "what is the best horror movie"],
        "best horror show": ["What is the highest rated horror show on netflix", "What is the highest rated horror show?", "highest rated horror show?", "best horror show", "scariest horror show", "what is the best horror show?", "what is the best horror show"],
 
        "worst horror movie": ["What is the lowest rated horror movie on netflix", "What is the lowest rated horror movie?", "lowest rated horror movie?", "worst horror movie", "what is the worst horror movie?", "what is the worst horror movie"],
       "worst horror show": ["What is the lowest rated horror show on netflix", "What is the lowest rated horror show?", "lowest rated horror show?", "worst horror show", "what is the worst horror show?", "what is the worst horror show"]
}
def find_best_ms(movie = True):
    "Finds the best movie or show"
    if movie == True:
        a = table.find({"type": "movie"}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') # returns a life on out planet
    else:
        a = table.find({"type": "show"}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') # returns breaking bad
            

def find_worst_ms(movie = True):
    "Finds the worst movie or show"
    if movie == True:
        a = table.find({"type": "movie"}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') # My Girl
    else:
        a = table.find({"type": "show"}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') # Love 101

            
def find_best_ms_g(movie = True):
    "Finds the genre of the best movie or show"
    if movie == True:
        a = table.find({"type": "movie"}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, main_genre: {x["main_genre"]}') # returns a life on out planet
    else:
        a = table.find({"type": "show"}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, main_genre: {x["main_genre"]}') # returns breaking bad
            

def find_worst_ms_g(movie = True):
    "Finds the genre of the worst movie or show"
    if movie == True:
        a = table.find({"type": "movie"}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, main_genre: {x["main_genre"]}') # My Girl
    else:
        a = table.find({"type": "show"}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, main_genre: {x["main_genre"]}') # Love 101
            
            
def find_worst_genre(genre,movie = True):
    "Finds the worst movie or show"
    if movie == True:
        a = table.find({"type": "movie", "main_genre": genre}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 
    else:
        a = table.find({"type": "show", "main_genre": genre}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 
            
def find_best_genre(genre,movie = True):
    "Finds the best movie or show of that genre"
    if movie == True:
        a = table.find({"type": "movie", "main_genre": genre}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 
    else:
        a = table.find({"type": "show", "main_genre": genre}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 
            
            
def find_best_2022(movie = True):
    "Finds the best movie or show of this year 2022"
    if movie == True:
        a = table.find({"type": "movie", "release_year": 2022}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 
    else:
        a = table.find({"type": "show", "release_year": 2022}).sort("score", -1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 
            
def find_worst_2022(movie = True):
    "Finds the worst movie or show of this year 2022"
    if movie == True:
        a = table.find({"type": "movie", "release_year": 2022}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 
    else:
        a = table.find({"type": "show", "release_year": 2022}).sort("score", 1).limit(1)
        for x in a:
            return(f'{x["title"]}, Score: {x["score"]}') 

In [316]:
search = [find_best_ms(), find_best_ms(False),find_worst_ms(), find_worst_ms(False), \
          find_best_2022(), find_best_2022(False), find_worst_2022(), find_worst_2022(False),\
          find_best_ms_g(), find_best_ms_g(False),find_worst_ms_g(), find_worst_ms_g(False), \
          find_best_genre("documentary"),find_best_genre("documentary", False), find_worst_genre("documentary"), find_worst_genre("documentary", False),\
          find_best_genre("scifi"),find_best_genre("scifi", False), find_worst_genre("scifi"), find_worst_genre("scifi", False),\
          find_best_genre("drama"),find_best_genre("drama", False), find_worst_genre("drama"), find_worst_genre("drama", False),\
          find_best_genre("comedy"),find_best_genre("comedy", False), find_worst_genre("comedy"), find_worst_genre("comedy", False),\
          find_best_genre("horror"),find_best_genre("horror", False), find_worst_genre("horror"), find_worst_genre("horror", False)]         

In [308]:

while True:
    m = input("bot: What's up?\nYou: ")
    if m == "q" or m == "quit":
        break
    if m == "h" or m == "help":
        print("VALID INPUTS:")
        for k, v in Q_L.items():
            print("\n",k, v,"\n")
        continue
        
    valid = False
    for k, v in Q_L.items():
        if m in v:
            num = list(Q_L).index(k)
            print(search[num])
            valid = True
    if valid == False:
        import random
        print("Try 'what is' instead of what's")
        print(f"No pattern. Try: '{random.choice(list(Q_L)).replace('_', ' ')}'")
        
    
    
    

bot: What's up?
You: what is the best horror movie
the exorcist, Score: 8.1
bot: What's up?
You: h
VALID INPUTS:

 best movie ['best movie', 'best movie?', 'What movie is the best?', 'Recommend me a movie'] 


 best show ['best show', 'best show?', 'What movie is the show?', 'Recommend me a show'] 


 worst movie ['worst movie', 'worst movie?', 'What movie is the worst?', 'bad movie', 'bad movie'] 


 worst show ['worst show', 'worst show?', 'What show is the worst', 'bad show', 'bad show?'] 


 best movie now ['What is the best movie out now?', 'best movie this year', 'best movie this year?', 'What movie is the best this year?', 'Recommend me a movie from this year', 'best movie now'] 


 best show now ['What is the best show out now?', 'best show this year', 'best show this year?', 'What show is best this year?', 'Recommend me a show from this year', 'best show now'] 


 worst movie now ['What is the worst movie out now?', 'worst movie this year', 'worst movie now?', 'What movie is t

KeyboardInterrupt: Interrupted by user